In [28]:
import pandas as pd
from datasets import load_dataset
import chromadb
from dotenv import load_dotenv
import os
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_groq import ChatGroq

# Load environment variables
load_dotenv()

# Load dataset
ds = load_dataset("entfane/psychotherapy")
df = pd.DataFrame(ds['train'])
df.drop('stage', axis=1, inplace=True)

RED_FLAGS = {
    "self-harm", 
    "suicidal ideation", 
    "addiction or substance use disorder", 
    "bipolar disorder", 
    "schizophrenia", 
    "borderline personality disorder", 
    "impulse control disorder", 
    "post-traumatic stress disorder"
}

# Extract conversations and emotions
conversation = ds['train']['conversations']
emotion = ds['train']['illness']

# Process conversations
def get_conversation_text(conversation):
    docs = []
    for i in range(len(conversation)):
        text = ""
        for j in range(len(conversation[i])):
            text += " ".join(conversation[i][j].values())
        docs.append(text)
    return docs

# Prepare data for Chroma
documents = get_conversation_text(conversation)
metadatas = [{"ilness": e} for e in emotion]
ids = [str(i) for i in range(len(emotion))]

# Set up Chroma client
chroma_client = chromadb.PersistentClient(path="chroma_db")
collection = chroma_client.get_or_create_collection(name="Test")

# Add data to Chroma
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids,
)

# Query Chromadb
results = collection.query(
    query_texts=["I feel anxious and tired"],
    n_results=1,
)

Using the latest cached version of the dataset since entfane/psychotherapy couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\elect\.cache\huggingface\datasets\entfane___psychotherapy\default\0.0.0\9069680aaad6d2e85f418d68efad4ecae876abc6 (last modified on Fri Apr 18 02:05:05 2025).


In [31]:
# Helper functions to extract results
def get_therapist_answer():
    if 'documents' in results and results['documents'] and len(results['documents'][0]) > 0:
        return results['documents'][0][0]
    else:
        print("Error: No documents found in ChromaDB results. Returning a default response.")
        return "I understand you're feeling anxious and tired. Let's explore these feelings together."

def get_emotion():
    if 'metadatas' in results and results['metadatas'] and len(results['metadatas'][0]) > 0 and 'ilness' in results['metadatas'][0][0]:
        return results['metadatas'][0][0]['ilness']
    else:
        print("Error: 'illness' not found in ChromaDB results. Returning a default value.")
        return "anxiety"  # Return a default value or handle the error as needed

# Check if emotion is a red flag
def verify_emotion(emotion):
    return emotion in RED_FLAGS

In [46]:
# Choose which LLM to use (uncomment the one you want to use)

# Option 1: Use Google's Gemini model
from crewai import Agent, LLM

llm = LLM(
    api_key=os.getenv("GOOGLE_API_KEY"),
    model="gemini/gemini-1.5-flash",
)


# Get current results
therapist_answer = get_therapist_answer()
emotion = get_emotion()

# Create agent
SuggestionAgent = Agent(
    role="Supportive Therapist",
    goal=f"Provide empathetic and helpful responses based on detected emotions: {emotion} using the therapist advice: {therapist_answer}",
    backstory=f"""You are a supportive therapist who tailors responses based on
    the emotional state of your clients. You're compassionate and insightful. The emotion of the client is: {emotion}""",
    memory=True,
    verbose=True,
    allow_delegation=True,
    llm=llm,
)

# Create task
suggestion_task = Task(
    description=(
        f"Based on the therapist's advice: {therapist_answer}, provide a brief, empathetic response with concise advice."
    ),
    expected_output="A single brief paragraph of supportive advice based on the therapist's guidance.",
    agent=SuggestionAgent,
    output_file="output.txt",  
)

# Create crew
crew = Crew(
    agents=[SuggestionAgent],
    tasks=[suggestion_task],
    share_crew=True,
    llm=llm,
    verbose=True,
)

In [47]:
# Execute the crew
try:
    result = crew.kickoff(inputs={
        "therapist_answer": therapist_answer,
        "emotion": emotion
    })
    print(result)
except Exception as e:
    print(f"An error occurred: {e}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7eaf4a5b-95d3-42a2-8a49-fe00628d18a0                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 57608319-bbfa-44d8-9fb2-7a872f16400a
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 57608319-bbfa-44d8-9fb2-7a872f16400a
       Status: Executing Task...
    └── 🤖 Agent: Supportive Therapist
            Status: In Progress

# Agent: Supportive Therapist
## Task: Based on the therapist's advice: client Okay, I guess that's time. I still feel a little anxious, but not as much as when we started. Thanks.therapist You're welcome. Remember the deep breathing exercises we practiced if the anxiety starts to creep back up. Even a few minutes can make a difference. How's your sleep been this week?client It's been... okay. Still waking up in the middle of the night sometimes, but I'm trying to get back to sleep instead of getting up and pacing. It works sometimes.therapist That's good to hear. Keep practicing that. We can explore other sleep strategies next week if needed. Remember, consistency is key. Have a good week, and I'll see you next session., provide a brief, empathetic response with concise advice.


# Agent: Supportive Therapist
## Final Answer: 
It sounds like you're making progress in managing your anxiety and sleep.  That's wonderful! Remember, even small steps forward are significant.  Keep practicin

🚀 Crew: crew
└── 📋 Task: 57608319-bbfa-44d8-9fb2-7a872f16400a
       Status: Executing Task...
    └── 🤖 Agent: Supportive Therapist
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 57608319-bbfa-44d8-9fb2-7a872f16400a
       Assigned to: Supportive Therapist
       Status: ✅ Completed
    └── 🤖 Agent: Supportive Therapist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 57608319-bbfa-44d8-9fb2-7a872f16400a                                                                     │
│  Agent: Supportive Therapist                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7eaf4a5b-95d3-42a2-8a49-fe00628d18a0                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

It sounds like you're making progress in managing your anxiety and sleep.  That's wonderful! Remember, even small steps forward are significant.  Keep practicing the deep breathing, and if the middle-of-the-night wakings persist, we'll explore other techniques together next week.  Focus on the progress you've already made – you're doing great.
